In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import os
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader

%load_ext tensorboard
from torch.utils.tensorboard import SummaryWriter

In [2]:
def window_data(data):
  X = []
  Y = []
  for d in range(data.shape[0]):
    if d >= dias_para_previsao:
      X.append(data[d - dias_para_previsao:d])
      Y.append(data[d])

  X = np.array(X).reshape(-1,dias_para_previsao)
  Y = np.array(Y).reshape(-1)
  
  return X,Y

def get_data_set():
  with open("./predicting-bitcoin-prices-using-LSTM/btc.csv", "r") as f:
    f.readline()
    data = [float(t.split(",")[5]) for t in f.readlines()]
  
  train_data = np.array(data)
  train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1)).reshape(-1)
  
  X,Y = window_data(train_data_normalized)
  X = torch.from_numpy(X).view(-1,dias_para_previsao, 1)
  X.requires_grad_()
  X = X.to(torch.float)
  Y = torch.from_numpy(Y).view(-1,1)
  Y.requires_grad_()
  Y = Y.to(torch.float)

  train_data_set = TensorDataset(X[:1018], Y[:1018])
  test_data_set = TensorDataset(X[1018:], Y[1018:])

  train_data_set_complete = DataLoader(train_data_set, batch_size=len(train_data_set))
  train_data_set = DataLoader(train_data_set, batch_size=batch_size, shuffle=True)
  test_data_set = DataLoader(test_data_set, batch_size=len(test_data_set))
  return train_data_set,train_data_set_complete,test_data_set

In [3]:
class LSVM_Bitcoin(nn.Module):
  def __init__(self, drop_prob=0.5):
    super().__init__()
    self.LSTM_layers = nn.LSTM(1,num_hidden,num_layers, batch_first=True, dropout = drop_prob)
    self.linear_layer1 = nn.Linear(dias_para_previsao*num_hidden,1)
    self.bn1 = nn.BatchNorm1d(num_features = dias_para_previsao*num_hidden)
    self.sigmoid = nn.Sigmoid()


  def forward(self, x):
    hidden_cell = (torch.zeros(num_layers,x.shape[0],num_hidden, requires_grad=True).to(torch.float).to(dev),
                  torch.zeros(num_layers,x.shape[0],num_hidden, requires_grad=True).to(torch.float).to(dev))
    
    lstm_out, hidden_cell = self.LSTM_layers(x, hidden_cell)
    out = lstm_out.reshape(x.shape[0], -1)
        
    out = self.sigmoid(self.linear_layer1(self.bn1(out)))
    return out

In [4]:
def get_model():
    model = LSVM_Bitcoin()
    model = model.to(dev)
    optimizer = optim.Adam(model.parameters(), lr = lr)
    return model, optimizer

In [5]:
def fit(train_data_set,train_data_set_complete,test_data):
    model, optimizer = get_model()
    loss_function = nn.MSELoss()

    print(model)

    for e in range(epoch):

        for x,y in train_data_set:
            x,y = x.to(dev), y.to(dev)
            
            optimizer.zero_grad()

            prediction = model(x)
            loss = loss_function(prediction,y)            
            loss.backward()
            optimizer.step()
        
        loss = Evaluate_loss(model,loss_function, train_data_set_complete, "Loss/train", e=e)
        
    loss = Evaluate_loss(model,loss_function, test_data, "Loss/test")
    Compare_graphs(model, train_data_set_complete, test_data)

In [6]:
def Evaluate_loss(model, loss_function, data_set, kind_data_set, e=0):
  for x,y in data_set:
    x = x.to(dev)
    y = y.to(dev)

    with torch.no_grad():
      prediction = model(x)
      loss = loss_function(prediction,y)
    graph.add_scalar(kind_data_set, loss, e)
    print("epoca " + str(e) + ": " + kind_data_set + ": " + str(loss), end = "\n")

    return loss

def Compare_graphs(model, train_data_set_complete, test_data):
  for x,y in train_data_set_complete:
    x = x.to(dev)
    y = y.to(dev)

    with torch.no_grad():
      prediction = model(x)
    for i in range(prediction.shape[0]):
      graph.add_scalars("Compare_graphs/train", {"Original": y[i], "Prediction": prediction[i]},i)

  for x,y in test_data:
    x = x.to(dev)
    y = y.to(dev)

    with torch.no_grad():
      prediction = model(x)
    for i in range(prediction.shape[0]):
      graph.add_scalars("Compare_graphs/test", {"Original": y[i], "Prediction": prediction[i]}, i)

In [ ]:
#Hiper Parametros:
batch_size=13
dias_para_previsao=15
epoch=150
lr = 8e-4

#Parametros:
num_layers= 5
num_hidden = 12

os.system("git clone https://github.com/brynmwangy/predicting-bitcoin-prices-using-LSTM.git")

graph = SummaryWriter()
scaler = MinMaxScaler(feature_range=(0, 1))

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
train_data_set,train_data_set_complete,test_data = get_data_set()

fit(train_data_set,train_data_set_complete,test_data)
graph.close()

In [ ]:
%tensorboard --logdir=./runs